In [7]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")


In [9]:
import sqlite3
conn = sqlite3.connect('loja.db')
cursor = conn.cursor()

In [11]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        color TEXT NOT NULL,
        price REAL NOT NULL
    )
""")
cursor.execute("INSERT INTO products (name, color, price) VALUES ('caneta','marrom',1.0)")
cursor.execute("INSERT INTO products (name, color, price) VALUES ('caneta','preta',1.5)")
cursor.execute("INSERT INTO products (name, color, price) VALUES ('caneta','amarela',3.0)")
cursor.execute("INSERT INTO products (name, color, price) VALUES ('caneta','verde',4.0)")
conn.commit()

In [12]:
sql = "SELECT id, name, color, price FROM products WHERE name='caneta' and color='verde'"
cursor.execute(sql)
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)

(4, 'caneta', 'verde', 4.0)
(8, 'caneta', 'verde', 4.0)


In [13]:
sql = "SELECT * FROM products"
cursor.execute(sql)
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)

(1, 'caneta', 'marrom', 1.0)
(2, 'caneta', 'preta', 1.5)
(3, 'caneta', 'amarela', 3.0)
(4, 'caneta', 'verde', 4.0)
(5, 'caneta', 'marrom', 1.0)
(6, 'caneta', 'preta', 1.5)
(7, 'caneta', 'amarela', 3.0)
(8, 'caneta', 'verde', 4.0)


In [14]:
cursor.close()
conn.close()

In [15]:
conn = sqlite3.connect('loja.db')
cursor = conn.cursor()
sql = "SELECT id, name, color, price FROM products WHERE name='caneta' and color='verde'"
cursor.execute(sql)
resultados = cursor.fetchall()
for linha in resultados:
    print(linha)
cursor.close()
conn.close()

(4, 'caneta', 'verde', 4.0)
(8, 'caneta', 'verde', 4.0)


In [18]:
def find_product(sql_query):
    conn = sqlite3.connect('loja.db')  # abrir no SQLiteStudio  https://sqlitestudio.pl/
    cursor = conn.cursor()
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute(sql_query)
    resultados = cursor.fetchall()
    results = [{"name": linha['name'], "color": linha['color'], "price": linha['price']} for linha in resultados]
    cursor.close()
    conn.close()
    return results


In [19]:
sql = "SELECT id, name, color, price FROM products WHERE name='caneta' and color='preta'"
resultados = find_product(sql)
print(resultados)

[{'name': 'caneta', 'color': 'preta', 'price': 1.5}, {'name': 'caneta', 'color': 'preta', 'price': 1.5}]


In [20]:
resultados[0]

{'name': 'caneta', 'color': 'preta', 'price': 1.5}

In [21]:
resultados[0]['name']

'caneta'

In [22]:
resultados[1]['price']

1.5

In [23]:
# Obtendo o schema da base de dados

from langchain_community.utilities import SQLDatabase
from pathlib import Path

db_string = f"sqlite:///loja.db"
db = SQLDatabase.from_uri(db_string, sample_rows_in_table_info=0)

def get_schema(_):
    return db.get_table_info()

print(get_schema(_))


CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	color TEXT NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)


In [24]:
# Obtenção do schema por outra forma

def get_schema(_):
    schemas = ''
    for table in db.get_usable_table_names():
        schemas += db.get_table_info([table]).split('/*')[0].strip()

    return schemas

print(get_schema(_))

CREATE TABLE products (
	id INTEGER, 
	name TEXT NOT NULL, 
	color TEXT NOT NULL, 
	price REAL NOT NULL, 
	PRIMARY KEY (id)
)
